In [1]:
import tqdm
import random
import pathlib
import itertools
import collections

import glob
import pandas as pd
import os
import cv2
import numpy as np
import remotezip as rz

import tensorflow as tf

# Some modules to display an animation using imageio.
import imageio
from IPython import display
from urllib import request
from tensorflow_docs.vis import embed

In [1]:
#########################
#########################
#########################

In [12]:
import random

# Create a directory to store all the clips
clips_dir = r"C:\Users\BEE Team\Desktop\code\Summarys_labels\All_Clips"
os.makedirs(clips_dir, exist_ok=True)

# Loop through all the CSV files
csv_dir = r'C:\Users\BEE Team\Desktop\code\Summarys_labels'
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
csv_files = csv_files[1:]

# Create the directories for train, validation, and test clips
train_dir = os.path.join(clips_dir, "train")
val_dir = os.path.join(clips_dir, "val")
test_dir = os.path.join(clips_dir, "test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Loop through all the CSV files
for csv_file in csv_files:
    if csv_file.endswith('.csv'):
        # Read the CSV file
        df = pd.read_csv(os.path.join(csv_dir, csv_file))

        # Directory of videos
        video_dir = r'C:\Users\BEE Team\Desktop\Videos\Daniele_Vids_Lab\LD\Yellow'
        df['Video_path'] = df['file'].apply(lambda x: os.path.join(video_dir, x.split('_labels')[0] + '.mp4'))

        # Create a directory to store the clips for this csv file
        csv_clips_dir = os.path.join(clips_dir, os.path.splitext(csv_file.split('_')[2])[0])
        os.makedirs(csv_clips_dir, exist_ok=True)

        # Extract video clips from each row of the dataframe and save them to a file
        for index, row in df.iterrows():
            video_path = row['Video_path']
            start_frame = row['start_frame']
            end_frame = row['end_frame']
            classification = row['classification']
            
            video = cv2.VideoCapture(video_path)
            video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
            clip_frames = []
            for i in range(start_frame, end_frame):
                ret, frame = video.read()
                if ret:
                    clip_frames.append(frame)
                else:
                    break
            
            video.release()
            
            clip_path = os.path.join(csv_clips_dir, f'{classification}_{index}.mp4')
            clip = cv2.VideoWriter(clip_path, cv2.VideoWriter_fourcc(*'XVID'), 30, (frame.shape[1], frame.shape[0]))
            for frame in clip_frames:
                clip.write(frame)
            clip.release()

            # Decide which directory to store the clip in based on the split
            rand_num = random.random()
            if rand_num < 0.6:
                dest_dir = os.path.join(train_dir, classification)
            elif rand_num < 0.8:
                dest_dir = os.path.join(val_dir, classification)
            else:
                dest_dir = os.path.join(test_dir, classification)

            # Move the clip to the appropriate directory
            os.makedirs(dest_dir, exist_ok=True)
            clip_name = os.path.basename(clip_path)
            dest_path = os.path.join(dest_dir, clip_name)
            os.rename(clip_path, dest_path)

In [21]:
def format_frames(frame, output_size):
  """
    Pad and resize an image from a video.

    Args:
      frame: Image that needs to resized and padded. 
      output_size: Pixel size of the output frame image.

    Return:
      Formatted frame with padding of specified output size.
  """
  frame = tf.image.convert_image_dtype(frame, tf.float32)
  frame = tf.image.resize_with_pad(frame, *output_size)
  return frame

In [22]:
def frames_from_video_file(video_path, n_frames, output_size = (224,224), frame_step = 15):
  """
    Creates frames from each video file present for each category.

    Args:
      video_path: File path to the video.
      n_frames: Number of frames to be created per video file.
      output_size: Pixel size of the output frame image.

    Return:
      An NumPy array of frames in the shape of (n_frames, height, width, channels).
  """
  # Read each video frame by frame
  result = []
  src = cv2.VideoCapture(str(video_path))  

  video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

  need_length = 1 + (n_frames - 1) * frame_step

  if need_length > video_length:
    start = 0
  else:
    max_start = video_length - need_length
    start = random.randint(0, max_start + 1)

  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  # ret is a boolean indicating whether read was successful, frame is the image itself
  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result

In [25]:
video_path = r"Summarys_labels\All_Clips\val\Int_Cleaner\Int_Cleaner_1.mp4"
sample_video = frames_from_video_file(video_path, n_frames = 10)
sample_video.shape

(10, 224, 224, 3)

In [26]:
def to_gif(images):
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images, fps=10)
  return embed.embed_file('./animation.gif')

In [28]:
#to_gif(sample_video)

In [30]:
video_dir = r'Summarys_labels\All_Clips\train\Int_Cleaner'

# get a list of all the video files in the directory
video_files = glob.glob(os.path.join(video_dir, '*.mp4'))

# select the first video file in the list
video_path = video_files[0]

# extract 50 frames from the selected video file
sample_videos = frames_from_video_file(video_path, 50)
#to_gif(sample_videos)

In [38]:
class FrameGenerator:
  def __init__(self, path, n_frames, training = False):
    """ Returns a set of frames with their associated label. 

      Args:
        path: Video file paths.
        n_frames: Number of frames. 
        training: Boolean to determine if training dataset is being created.
    """
    self.path = path
    self.n_frames = n_frames
    self.training = training
    self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

  def get_files_and_class_names(self):
    video_paths = list(self.path.glob('*/*.mp4'))
    classes = [p.parent.name for p in video_paths] 
    return video_paths, classes

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))

    if self.training:
      random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path, self.n_frames) 
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label

In [32]:
from pathlib import Path

In [51]:
fg = FrameGenerator(Path('Summarys_labels/All_Clips/train'), 10, training=True)

frames,label = next(fg())

print(f"Shape: {frames.shape}")
print(f"Label: {label}")

Shape: (10, 224, 224, 3)
Label: 1


In [52]:
# Create the training set
output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))
train_ds = tf.data.Dataset.from_generator(FrameGenerator(Path('Summarys_labels/All_Clips/train'), 10, training=True),
                                          output_signature = output_signature)

In [53]:
for frames, labels in train_ds.take(10):
  print(labels)

tf.Tensor(1, shape=(), dtype=int16)
tf.Tensor(2, shape=(), dtype=int16)
tf.Tensor(1, shape=(), dtype=int16)
tf.Tensor(0, shape=(), dtype=int16)
tf.Tensor(1, shape=(), dtype=int16)
tf.Tensor(1, shape=(), dtype=int16)
tf.Tensor(4, shape=(), dtype=int16)
tf.Tensor(2, shape=(), dtype=int16)
tf.Tensor(2, shape=(), dtype=int16)
tf.Tensor(2, shape=(), dtype=int16)


In [54]:
# Create the validation set
val_ds = tf.data.Dataset.from_generator(FrameGenerator(Path('Summarys_labels/All_Clips/val'), 10),
                                        output_signature = output_signature)

In [55]:
# Print the shapes of the data
train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

val_frames, val_labels = next(iter(val_ds))
print(f'Shape of validation set of frames: {val_frames.shape}')
print(f'Shape of validation labels: {val_labels.shape}')

Shape of training set of frames: (10, 224, 224, 3)
Shape of training labels: ()
Shape of validation set of frames: (10, 224, 224, 3)
Shape of validation labels: ()


# Configure the dataset for performance

In [56]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)

In [57]:
train_ds = train_ds.batch(2)
val_ds = val_ds.batch(2)

train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

val_frames, val_labels = next(iter(val_ds))
print(f'Shape of validation set of frames: {val_frames.shape}')
print(f'Shape of validation labels: {val_labels.shape}')

Shape of training set of frames: (2, 10, 224, 224, 3)
Shape of training labels: (2,)
Shape of validation set of frames: (2, 10, 224, 224, 3)
Shape of validation labels: (2,)


# Next steps

In [ ]:
net = tf.keras.applications.EfficientNetB0(include_top = False)
net.trainable = False

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(scale=255),
    tf.keras.layers.TimeDistributed(net),
    tf.keras.layers.Dense(10),
    tf.keras.layers.GlobalAveragePooling3D()
])

model.compile(optimizer = 'adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])

model.fit(train_ds, 
          epochs = 10,
          validation_data = val_ds,
          callbacks = tf.keras.callbacks.EarlyStopping(patience = 2, monitor = 'val_loss'))

In [ ]:
#########################
#########################
#########################